Load in data and check class ratio

In [47]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [58]:
canc = pd.read_csv("cancer_uci.csv", index_col=0)
canc.head()

,Sample_code_number,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,Benign
1,1002945,5,4,4,5,7,10,3,2,1,Benign
2,1015425,3,1,1,1,2,2,3,1,1,Benign
3,1016277,6,8,8,1,3,4,3,7,1,Benign
4,1017023,4,1,1,3,2,1,3,1,1,Benign


In [59]:
canc.Class = canc.Class.map({'Benign':0,'Malignant':1})
canc.Class.value_counts()

0    458
1    241
Name: Class, dtype: int64

In [60]:
# Separate your two classes:
mal_example = canc[canc.Class == 1] # give me the dataframe when Class = 1 
benign_example = canc[canc.Class == 0]

# Oversample the malignant class to have a 50/50 ratio:
mal_over_example = mal_example.sample(458,replace=True)

# Recombine the two frames:
over_sample = pd.concat([mal_over_example,benign_example])

# Sanity check the length:
print len(over_sample)

916


In [61]:
over_sample.head()

,Sample_code_number,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
421,1257200,10,10,10,7,10,10,8,2,1,1
286,529329,10,10,10,10,10,10,4,10,10,1
283,492268,10,4,6,1,2,10,5,3,1,1
246,144888,8,10,10,8,5,10,7,8,1,1
18,1050670,10,7,7,6,4,10,4,1,2,1


Obviously the data is not normalized, as there are values that are outside of the 0-1 range. 

In [62]:
# Normalize data
# taking out ID column
over_sample = over_sample.drop('Sample_code_number', axis=1)
over_sample.head()

,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
421,10,10,10,7,10,10,8,2,1,1
286,10,10,10,10,10,10,4,10,10,1
283,10,4,6,1,2,10,5,3,1,1
246,8,10,10,8,5,10,7,8,1,1
18,10,7,7,6,4,10,4,1,2,1


In [65]:
print over_sample.dtypes
over_sample.Bare_Nuclei.unique

Clump_Thickness                 int64
Uniformity_of_Cell_Size         int64
Uniformity_of_Cell_Shape        int64
Marginal_Adhesion               int64
Single_Epithelial_Cell_Size     int64
Bare_Nuclei                    object
Bland_Chromatin                 int64
Normal_Nucleoli                 int64
Mitoses                         int64
Class                           int64
dtype: object


<bound method Series.unique of 421    10
286    10
283    10
246    10
18     10
175    10
200    10
65      2
530    10
697     4
304    10
67     10
39      7
43      1
427     2
42     10
610    10
183    10
612    10
489     4
266    10
253    10
697     4
412     4
449    10
107    10
349     8
381    10
109     9
523    10
       ..
660     1
661     1
662     1
663     1
664     1
665     1
666     1
667     1
671     1
672     1
673     1
674     1
675     1
676     1
677     1
678     1
679     1
682     1
683     1
684     1
685     1
686     1
687     1
688     1
689     1
690     1
692     1
693     1
694     2
695     1
Name: Bare_Nuclei, dtype: object>

In [63]:
# converting to Numpy array
over_sample_array = over_sample.values
# normalizing data
over_sample_arraynorm = preprocessing.normalize(over_sample_array)
over_sample_arraynorm[:10,:]

ValueError: could not convert string to float: ?

In [56]:
over_sample.head()

,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
98,9,6,9,2,10,6,2,9,10,1
357,8,10,10,7,10,10,7,3,8,1
52,10,3,6,2,3,5,4,10,2,1
142,9,5,5,4,4,5,4,3,3,1
151,7,2,4,1,6,10,5,4,3,1


In [53]:
over_sample.values

array([[9L, 6L, 9L, ..., 9L, 10L, 1L],
       [8L, 10L, 10L, ..., 3L, 8L, 1L],
       [10L, 3L, 6L, ..., 10L, 2L, 1L],
       ..., 
       [3L, 1L, 1L, ..., 1L, 2L, 0L],
       [3L, 1L, 1L, ..., 1L, 1L, 0L],
       [2L, 1L, 1L, ..., 1L, 1L, 0L]], dtype=object)